# Отработка Хи квадрат

# Отличие хи квадрата Пирсона от той же корреляции - корреляция это линейная зависимость между данными  не категориальными !!!
# Хи квадрат позволяет находить взаимосвязь между категориальными данными

In [36]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency as stats

Описание данных
– respondentID — идентификационный номер респондента
– seen — ответ на вопрос, видел ли респондент хотя бы один фильм из серии «Звездных войн» (да/нет)
– fan — ответ на вопрос, считает ли респондент себя фанатом серии фильмов (да/нет)
– gender — пол респондента (М/Ж)
– age_group — возрастная группа, к которой принадлежит респондент (18-29, 30-44, 45-60, > 60)
– income — доход респондента в долларах США (0 - 24.999, 25.000 - 49.999, 50.000 - 99.999, 100.000 - 149.999, 150.000+)
– loc — регион проживания респондента (в США)
– yoda — отношение респондента к персонажу серии (Йоде)
– princess_leia_organa — отношение респондента к персонажу серии (принцессе Лее Органе)

In [9]:
df = pd.read_csv(filepath_or_buffer='../dataSet/StarWars.csv', sep=';')
df.head()

,respondentID,seen,fan,gender,age_group,income,loc,yoda,princess_leia_organa
0,3292879998,да,да,М,18-29,NaN,Южно-Атлантический,очень нравится,очень нравится
1,3292879538,нет,NaN,М,18-29,0 - 24.999,Юго-Западный центральный,NaN,NaN
2,3292765271,да,нет,М,18-29,0 - 24.999,Северо-Западный центральный,не знаю такого персонажа,нравится
3,3292763116,да,да,М,18-29,100.000 - 149.999,Северо-Западный центральный,очень нравится,очень нравится
4,3292731220,да,да,М,18-29,100.000 - 149.999,Северо-Западный центральный,нравится,нравится


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   respondentID          1186 non-null   int64 
 1   seen                  1186 non-null   object
 2   fan                   836 non-null    object
 3   gender                1046 non-null   object
 4   age_group             1046 non-null   object
 5   income                858 non-null    object
 6   loc                   1043 non-null   object
 7   yoda                  826 non-null    object
 8   princess_leia_organa  831 non-null    object
dtypes: int64(1), object(8)
memory usage: 83.5+ KB


1 Задание построить таблицу сопряженности для переменных gender, fan

In [16]:
df_gender_fan = df[(df['gender'].notnull()) & (df['fan'].notnull())].groupby('gender')['fan'].value_counts().unstack()
df_gender_fan

fan,да,нет
gender,,
Ж,238,159
М,303,120


Укажите, верно ли следующее утверждение: 120 мужчин являются фанатами серии фильмов «Звездные войны».

In [18]:
df_gender_fan['да'].iloc[1] == 120

False

Предположим, что мы тестируем гипотезу о том, что распределение наблюдений по категориям выбранных нами переменных независимо. Чему будет равно количество степеней свободы, рассчитанное по построенной в предыдущем задании таблице сопряженности? Выберите один верный ответ.
формула = (столбцов -1 ) * (строк - 1)

In [22]:
(df_gender_fan.columns.size - 1) * (df_gender_fan.index.size - 1)

1

Чему равна оценка вероятности того, что человек является фанатом серии фильмов? Введите ответ в виде десятичной дроби, округлите до двух знаков после запятой.

In [27]:
df_1 = df[(df['gender'].notnull()) & (df['fan'].notnull())]
fan_starWars = f'''{df_1[df_1['fan'] == 'да'].gender.size / df_1.gender.size:.2f}'''
fan_starWars

'0.66'

Чему равна теоретическая частота для женщин, которые являются фанатами серии фильмов? В ответ запишите число, округлите до целого.

In [41]:
# воспользуемся scipy
hi = stats(df_gender_fan,correction=False)
hi

(12.449350493035466,
 0.00041813719910750865,
 1,
 array([[261.92317073, 135.07682927],
        [279.07682927, 143.92317073]]))

In [37]:
# попробуем посчитать сами

# 1 шаг - вычисляем общее количество наблюдений
total_human = df_gender_fan.sum().sum()
# 2 шаг - общее количество фанатов/ не фанатов
fans = df_gender_fan['да'].sum()
not_fans = df_gender_fan['нет'].sum()
# 3 шаг рассчитываем вероятность того, что человек является фанатом и не фанатом
ver_fan = fans / total_human
ver_not_fan = not_fans / total_human
# 4 шаг посчитаем общее количество мужчин/женщин
n_men = df_gender_fan.loc['М'].sum()
n_women = df_gender_fan.loc['Ж'].sum()
# 5 шаг подсчитаем вероятности и сделаем из них таблицу
teoretical_ver_women_fan = n_women * ver_fan
teoretical_ver_women_not_fan = n_women * ver_not_fan
teoretical_ver_men_fan = n_men * ver_fan
teoretical_ver_men_not_fan = n_men * ver_not_fan
teor_arr = np.array([[teoretical_ver_women_fan,teoretical_ver_women_not_fan],[teoretical_ver_men_fan,teoretical_ver_men_not_fan]])
teor_arr

array([[261.92317073, 135.07682927],
       [279.07682927, 143.92317073]])

In [40]:
# еще один способ подсчета частоты это умножать строку на столбец и делить на общее число людей, рассчитаем так частоту для фанатов женщин
df_gender_fan.loc['Ж'].sum() * df_gender_fan['да'].sum()/ total_human
# сам подсчет хи квадрата
((teor_arr-df_gender_fan)**2/teor_arr).sum().sum()

12.449350493035455

Нулевую гипотезу мы отвергаем тк хи квадрат > уровня значимости в 5 %